In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import qutip as qt
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import time
import qiskit
from itertools import product
from typing import List, Dict, Tuple

In [6]:
qt.__version__

'5.1.1'

In [7]:
X = qt.sigmax()
Y = qt.sigmay()
Z = qt.sigmaz()
I = qt.identity(2)
Had = qt.core.gates.snot()
q0 = qt.basis(2,0)
q1 = qt.basis(2,1)
pl = Had*q0
pi = np.pi
sqrt=np.sqrt

### Functions for QAOA Grover's:
- phase_operator(C,gamma) : is a unitary generated by an operator C, in standard Grover's approach it is a projector to a bitstring.
- mixer(beta, NQ) is a standard QAOA mixer, for the Grover's beta angles are set to be $\pi/N$, where $N=$number of qubits
- W_op(C, gamma, NQ) is the total single block of the QAOA grover's

In [3]:
def phase_operator(C, gamma):
    return (-1j*C*gamma).expm()

def mixer(beta, NQ):
    M = [(-1j*beta*X).expm()]*NQ
    return qt.tensor(M)

def W_op(C, gamma, NQ):
    beta = np.pi/NQ
    W = mixer(beta, NQ ) * phase_operator(C, -gamma)*mixer(beta, NQ ) * phase_operator(C, gamma)
    return W

In [8]:
def hamiltonian_ising(Jij_dict, h, NQ = 3):
    H = qt.tensor([I]*NQ)*0
    for edge, Jij in Jij_dict.items():
        Z_temp = [I]*NQ
        Z_temp[edge[0]] = Z
        Z_temp[edge[1]] = Z
        H += Jij * qt.tensor(Z_temp)
    for idx in range(NQ):
        Z_local = [I] * NQ
        Z_local[idx] = Z
        H += h[idx] * qt.tensor(Z_local)
    return H

In [ ]:
q0 = qt.basis(2)
q1 = qt.basis(2,1)
q_pl = 1/np.sqrt(2)*(q0+q1)


class QAOA:
    '''
    A class for solving QAOA problem as the input it takes the J_ij in the form of a dictionary,
    and a list (length NQ - number of qubits) representing local fields (h_i), that creates
    a cost Hamiltonian

    '''
    
    
    def __init__(self, J_ij, h):
        '''
        initialization of quantities that we keep track of:
        angles
        cost function values
        separate gamma and beta angles (you can also read it off the angles)
        it - iterations until the optimization has stopped
        tin - time for the optimization
        spectral_gap - a list of [ground state eng, first excited state, gap ]
                       here the first exicted state is the first state with different energy than the GS
        
        '''
        self.h = h
        self.J_ij = J_ij
        self.NQ = len(self.h)
        self.H = hamiltonian_ising(J_ij,h, NQ )
        
        self.angles = []
        self.cost_value = []
        self.gamma_list_opt = []
        self.beta_list_opt = []
#         self.p_layers = p_layers
        self.it = 0
        self.tin = 0
#         self.constraints = constraints
        EV = list(set(self.H.eigenenergies()))
        EV.sort()
        self.spectral_gap = [EV[0], EV[1], abs(EV[0]-EV[1])]
        

    def cost(self, angles_list, psi_in = q_pl):

        self.it += 1
        if self.it % 250 == 0:
            print(f'it : {self.it}, time : {time.monotonic() - self.tin}')
#         print(f'it : {self.it}')
        if len(psi_in.dims[0]) == 1:
            psi = qt.tensor([psi_in]*NQ)
        else:
            psi = psi_in
        p = len(angles_list)
        gamma_list = angles_list[:int(p/2)]
        beta_list = angles_list[int(p/2):]
        U_circ = qt.tensor([I]*self.NQ)


        for layer, (gamma, beta) in enumerate(zip(gamma_list, beta_list)):

            Ux = mixer(beta, self.NQ)
            Up = (-1j*gamma*self.H).expm()
            U_circ *= Ux*Up

            psi_p = U_circ * psi
            psi_p_dict[layer] = psi_p
        psi_out = U_circ*psi
        
        C = qt.expect(self.H, psi_out)
        self.angles.append(angles_list)
        self.gamma_list_opt.append(gamma_list)
        self.beta_list_opt.append(beta_list)
        self.cost_value.append(C)
            
        return C

            
    def optimize(self, x0, method, bounds = [0], double = False):
        '''
        Input : 
        x0     - initial angles for the optimization, they are structured as:
                 length of the array is 2p length (p is the number of layers),
                 the first half corresponds to the gamma angles, the second to the beta
        method - optimization method, same as in scipy.optimize.minimize routine
        boudns - if [0] no bounds are employed, otherwise you need to specify a nested list/tuple
                 of size 2x2p, i.e. p gamma min and max angles and p beta min/max angles
        '''
        self.tin = time.monotonic()
#         print(f'tin : {tin}')
        if bounds[0] == 0:
            res = minimize(self.cost, x0, method = method)
        else:
            res = minimize(self.cost, x0, method = method, bounds= bounds)
        print(f'time ellapsed for optimization : {time.monotonic() - self.tin}')
        return res           
    
    def plot_cost(self, save = False, path = '', show = True):
        it_list = np.arange(len(self.cost_value))
        name = 'Dark2'
        cmap = get_cmap(name)  # type: matplotlib.colors.ListedColormap
        colors = cmap.colors  # type: list
        c = colors[0]
        min_cost = min(self.H.eigenenergies())
        
        plt.rcParams.update({'font.size':40})
        plt.figure(100,figsize=(24,16))
        plt.grid(True)
        plt.plot(it_list, self.cost_value, linewidth = 5, c=c)
        plt.plot(it_list, [min_cost]*self.it, c = 'black', linewidth = 5, linestyle = 'dashed', label = 'min')
        plt.legend()
        plt.xlabel(r'it')
        plt.ylabel(r'$\langle H\rangle$')
#         plt.legend()
        if save:
            path_save = './Plots/' + path
            if not os.path.exists(path_save):
                os.mkdir(path_save)
            plt.savefig(path_save + '/cost.pdf')
        if show:
            plt.show()
        plt.clf()
    
    def plot_cost_diff(self, style = 'lin', save = False, path = '', show = True):
        it_list = np.arange(len(self.cost_value))
        name = 'Dark2'
        cmap = get_cmap(name)  # type: matplotlib.colors.ListedColormap
        colors = cmap.colors  # type: list
        c = colors[0]
        min_cost = min(self.H.eigenenergies())

        plt.rcParams.update({'font.size':40})
        plt.figure(100,figsize=(24,16))
        plt.grid(True)
        if style == 'lin':
            plt.plot(it_list, self.cost_value-min_cost, linewidth = 5, c=c)
        elif style == 'log':
            plt.semilogy(it_list, self.cost_value-min_cost, linewidth = 5, c=c)
        #         plt.legend()
        plt.xlabel(r'it')
        plt.ylabel(r'$\langle H\rangle$')
#         plt.legend()
        if save:
            path_save = './Plots/' + path
            if not os.path.exists(path_save):
                os.mkdir(path_save)
            plt.savefig(path_save + '/cost_diff.pdf')
        if show:
            plt.show()
        plt.clf()

        
        
    def plot_angles(self, save = False, path = '', show = True):
        it_list = np.arange(self.it)
        name = 'Dark2'
        cmap = get_cmap(name)  # type: matplotlib.colors.ListedColormap
        colors = cmap.colors  # type: list
        
        plt.rcParams.update({'font.size':45})
        fig, ax = plt.subplots(2,1, figsize=(30,50),sharex=True)
        g_list = np.array(self.gamma_list_opt).T
        b_list = np.array(self.beta_list_opt).T
        for idx, (gamma, beta) in enumerate(zip(g_list,b_list)):
            c = colors[idx]
            ax[0].plot(it_list, gamma, linewidth = 5, c=c, label = f'p = {idx+1}')
            ax[1].plot(it_list, beta, linewidth = 5, c=c, label = f'p = {idx+1}')
        ax[1].set_xlabel(r'it')
        ax[0].set_ylabel(r'$\gamma$')
        ax[1].set_ylabel(r'$\beta$')
        
        if save:
            path_save = './Plots/' + path
            if not os.path.exists(path_save):
                os.mkdir(path_save)
            plt.savefig(path_save + '/angles.pdf')
#         plt.legend()
        if show:
            plt.show()
        plt.clf()
        
    def save_function(self, file_name, path):
        '''
        function that saves as a pickle file all values that we keep track off
        '''
        if not os.path.exists(path):
            os.mkdir(path)
            
        save_dict = {'H' : self.H, 
                     'spectrum' : self.spectral_gap,
                     'cost' : self.cost_value,
                     'it' : self.it,
                     'angles' : self.angles,
                    'h' : self.h, 
                    'J_ij' : self.J_ij} 

        with open(path +  file_name, 'wb') as f:
            pickle.dump(save_dict, f)  
        